In [1]:
from tqdm.notebook import tqdm
import numpy as np
import sys
sys.path.append('../')
from partitioning import is_pow_of_two, init_membership_list
# from spawn import start_db_background
from structures import Params, NetworkParams
from client_dynamo import client_get, client_put, client_get_memory
from parallel_runner import run_parallel
from random import randint

import logging
logger = logging.getLogger('dynamo_node')
logger.setLevel(logging.INFO)

START_PORT = 2333
CLIENT_ID = 1

def get_start_port(randomize=True):
    return START_PORT + randint(0, params.num_proc-1) * int(randomize)

def get_stats(durations):
    if len(durations) == 0:
        return {}
    durations = np.array(durations) # convert from seconds to ms
    mean = np.mean(durations)
    std = np.std(durations)
    nnth = np.percentile(durations, 99.9)
    return {'mean': mean, 'std': std, '99.9th': nnth}

def generate_plot(durations, label='', clear=True):
    if clear:
        plt.clf()
    fig = sns.distplot(durations, label=label)
    plt.ylabel('Density')
    plt.xlabel('Response Time (in ms)')
    plt.title('Distribution of response times (in ms)')
    plt.legend()
    plt.show()


In [2]:
params = Params({
    'num_proc' : 8,
    'hash_size': 8, # 2^3 = 8 
    'Q' : 16, # 
    'N' : 4,
    'w_timeout': 2,
    'r_timeout': 2,
    'R': 1,
    'W': 3,
    'gossip': False
})

In [3]:
def run_multiple_get(total, num_requests, get_durations):
    for i in tqdm(range(total // num_requests)):
        requests = [client_get]*num_requests
        requests_params = [{'port': get_start_port(), 'client_id': CLIENT_ID, 'key': randint(0, 2**params.hash_size-1)} for _ in range(num_requests)]
        get_durations = np.concatenate((get_durations, run_parallel(requests, requests_params, start_port=START_PORT)))
    return get_durations

def run_multiple_put(total, num_requests, put_durations):
    for i in tqdm(range(total // num_requests)):
        requests = [client_put]*num_requests
        k = randint(0, 2**params.hash_size-1)
        requests_params = [{'port': get_start_port(), 'client_id': CLIENT_ID, 'key': k, 'val': str(k)} for _ in range(num_requests)]
        put_durations = np.concatenate((put_durations, run_parallel(requests, requests_params, start_port=START_PORT)))
    return put_durations

def store_keys(params):
    for key in tqdm(range(2**params.hash_size)):
        client_put(get_start_port(), CLIENT_ID, key=key, val=str(key))

In [4]:
store_keys(params)

## With failures

In [ ]:
import logging
logger = logging.getLogger('dynamo_node')
logger.setLevel(logging.INFO)
# logger.propagate = False

In [6]:
def get_start_port(randomize=True, failed_port=None):
    new_port = START_PORT + randint(0, params.num_proc-1) * int(randomize)
    while failed_port is not None and new_port == failed_port:
        new_port = START_PORT + randint(0, params.num_proc-1) * int(randomize)
    return new_port

def run_multiple_get(total, num_requests, get_durations=None, failed_port=None):
    durations, responses = [], []
    for i in tqdm(range(total // num_requests)):
        requests = [client_get]*num_requests
        key = randint(0, 2**params.hash_size-1)
        while (key >= 176 and key <= 191) or (key >= 208 and key <= 223):
            # dont want a key to go to a failed coordinator node
            key = randint(0, 2**params.hash_size-1)                             
        requests_params = [{'port': get_start_port(failed_port=failed_port), 'client_id': CLIENT_ID, 'key': key} for _ in range(num_requests)]
        _durations, _responses = run_parallel(requests, requests_params, start_port=START_PORT, as_np=False)
        print(len(_durations))
        durations.extend(_durations)
        responses.extend(_responses)
    return durations, responses

In [5]:
from client_dynamo import client_fail

START_PORT
mem1, replic_mem = client_get_memory(START_PORT)

keys = []
for k,v in mem1.items():
    keys.append(k)

sorted(keys)

[176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223]

In [7]:
client_fail(START_PORT)

In [ ]:
response = client_put(START_PORT+1, CLIENT_ID, 0, "0")
print(response)

response = client_get(START_PORT+1, CLIENT_ID, 0)
print(response)

In [8]:
# get_fail_durations = np.array([])
get_fail_durations, responses = run_multiple_get(30, 10, None, START_PORT)

10 [0.03500699996948242, 0.02516317367553711, 0.025689125061035156, 0.030003070831298828, 0.02937483787536621, 0.03000497817993164, 0.032386064529418945, 0.027057886123657227, 0.02564406394958496, 0.024981021881103516]
10
10 [0.0274050235748291, 0.03005218505859375, 0.01553487777709961, 0.012842893600463867, 0.04655575752258301, 0.014355897903442383, 0.029096126556396484, 0.028202056884765625, 0.02606797218322754, 0.030539989471435547]
10
10 [0.014088869094848633, 0.030475139617919922, 0.030516862869262695, 0.030209064483642578, 0.04502105712890625, 0.013289928436279297, 0.026469945907592773, 0.027759075164794922, 0.028607845306396484, 0.028111934661865234]
10



In [9]:
get_fail_durations = np.array(get_fail_durations)

get_fail_durations.shape

(30,)

In [10]:
len(get_fail_durations)

30

In [11]:
get_fail_durations*1000

array([35.00699997, 25.16317368, 25.68912506, 30.00307083, 29.37483788,
       30.00497818, 32.38606453, 27.05788612, 25.64406395, 24.98102188,
       27.40502357, 30.05218506, 15.53487778, 12.8428936 , 46.55575752,
       14.3558979 , 29.09612656, 28.20205688, 26.06797218, 30.53998947,
       14.08886909, 30.47513962, 30.51686287, 30.20906448, 45.02105713,
       13.28992844, 26.46994591, 27.75907516, 28.60784531, 28.11193466])

In [ ]:
len(responses)

In [ ]:
get_durations.shape

In [ ]:
test = [0.029925823211669922, 0.03148627281188965, 0.031996965408325195, 0.03176593780517578, 0.031462907791137695, 0.030453920364379883, 0.03252911567687988, 0.0177609920501709, 0.1620798110961914, 0.2121727466583252, 0.22617697715759277, 0.2506279945373535, 0.24697089195251465, 0.29291319847106934, 0.4482302665710449, 0.40848278999328613, 0.48263120651245117]

In [ ]:
len(test)